# Most-stable allocation under distributional constraints
### Kirill Zakharov
2021

In [48]:
from gurobipy import *
import numpy as np
import random

### Introduce quotas and dimensional information

In [174]:
# Defining Sets
n = 25
m = 5
p = 3 #number of types

students = [f"st {i}" for i in range(n)]
jobs = [f"comp {i}" for i in range(m)]

#quotas
upperQ = list(np.random.randint(7, 8, size=m))
lowerQ = list(np.random.randint(4, 5, size=m))

lb_of_types = [np.random.randint(1, 2, size=p) for j in range(m)]
ub_of_types = [np.random.randint(4, 5, size=p) for j in range(m)]

applicants_types = np.random.randint(1, 4, size=n)

In [175]:
ub_of_types[0]

array([4, 4, 4])

In [176]:
#Students' ranks of companies
a_ranks = [random.sample(list(np.arange(m)+1), m) for i in range(n)]

#Companies' scores of students
c_score = np.array([np.random.randint(1, 10, size=n) for j in range(m)]).T

In [177]:
model= Model("Assignment Model")

### Introduce deficiency variables and binary variables

In [178]:
#Defining the Variable
X = {}
for i in range(n):
    for j in range(m):
        X[i,j] = model.addVar(vtype= GRB.BINARY)
        
d = {}
for i in range(n):
    for j in range(m):
        d[i,j] = model.addVar(lb=0.0, ub=float('inf'), vtype= GRB.CONTINUOUS)

### Define objective function as sum of deficiency variables

In [179]:
#Objective Function
# model.setObjective(quicksum(a_ranks[i][j]*X[i,j] for i in range(n) for j in range(m)), GRB.MINIMIZE)
model.setObjective(quicksum(d[i,j] for i in range(n) for j in range(m)), GRB.MINIMIZE)

## First type of constraints

### Define constraints

In [180]:
#Constraint-1
for i in range(n):
     model.addConstr(quicksum(X[i,j] for j in range(m)) <= 1)
#Constraint-2
for j in range(m):
    model.addConstr(quicksum(X[i,j] for i in range(n)) <= upperQ[j])
    
#Constraint-3
for j in range(m):
    model.addConstr(quicksum(X[i,j] for i in range(n)) >= lowerQ[j])        

# for i in range(n):
#     for j in range(m):
#         model.addConstr(quicksum(X[i,k] for k in range(m) if (a_ranks[i][k] <= a_ranks[i][j]))*upperQ[j] \
#                         + quicksum(X[h,j] for h in range(n) if c_score[h][j] > c_score[i][j]) >= upperQ[j])
#Constraint-4
for i in range(n):
    for j in range(m):
        model.addConstr(quicksum(X[i,k] for k in range(m) if (a_ranks[i][k] <= a_ranks[i][j]))*upperQ[j] \
                        + quicksum(X[h,j] for h in range(n) if c_score[h][j] >= c_score[i][j]) + d[i,j] \
                        >= upperQ[j])
#Constraint-5    
for i in range(n):
    for j in range(m):
        model.addConstr(X[i,j] >= 0)
        
#Constraint-6
for j in range(m):
    for l in range(p):
        model.addConstr(quicksum(X[i,j] for i in range(n) if applicants_types[i] == l+1) <= ub_of_types[j][l])
        
#Constraint-7
for j in range(m):
    for l in range(p):
        model.addConstr(quicksum(X[i,j] for i in range(n) if applicants_types[i] == l+1) >= lb_of_types[j][l])

### Solution

In [181]:
model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 315 rows, 250 columns and 2926 nonzeros
Model fingerprint: 0xad08b156
Variable types: 125 continuous, 125 integer (125 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+00]
Found heuristic solution: objective 305.0000000
Presolve removed 127 rows and 2 columns
Presolve time: 0.01s
Presolved: 188 rows, 248 columns, 2797 nonzeros
Variable types: 0 continuous, 248 integer (127 binary)

Root relaxation: objective 0.000000e+00, 155 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   27  305.00000    0.00000   100%     -    0s
H    0     0                      33.0000000    0.

In [182]:
res1 = np.array(model.X[:125]).reshape((n,m))
res1

array([[-0., -0.,  1., -0., -0.],
       [ 0., -0., -0.,  1.,  0.],
       [-0., -0., -0., -0.,  1.],
       [ 1., -0., -0., -0., -0.],
       [-0., -0., -0., -0.,  1.],
       [ 1., -0., -0., -0., -0.],
       [-0., -0., -0.,  1., -0.],
       [-0., -0., -0.,  1., -0.],
       [-0.,  1., -0., -0., -0.],
       [-0., -0.,  1., -0., -0.],
       [-0., -0., -0., -0.,  1.],
       [-0., -0., -0., -0.,  1.],
       [-0., -0.,  1., -0., -0.],
       [-0., -0.,  1., -0., -0.],
       [ 1., -0., -0., -0., -0.],
       [-0.,  0.,  1., -0., -0.],
       [-0.,  1., -0., -0., -0.],
       [-0.,  1.,  0., -0.,  0.],
       [ 1., -0.,  0., -0., -0.],
       [ 1., -0., -0., -0., -0.],
       [-0.,  0.,  1.,  0., -0.],
       [-0., -0., -0.,  1., -0.],
       [-0.,  1., -0., -0.,  0.],
       [-0., -0., -0., -0.,  1.],
       [-0., -0., -0.,  0.,  1.]])

In [183]:
a_ranksTable = np.array(a_ranks).reshape((n,m))
a_ranksTable

array([[2, 5, 1, 4, 3],
       [3, 4, 5, 2, 1],
       [4, 3, 5, 2, 1],
       [1, 3, 2, 5, 4],
       [4, 5, 2, 3, 1],
       [1, 4, 3, 2, 5],
       [2, 5, 4, 1, 3],
       [2, 3, 5, 1, 4],
       [5, 1, 2, 3, 4],
       [3, 5, 1, 4, 2],
       [5, 2, 4, 3, 1],
       [3, 4, 2, 5, 1],
       [4, 5, 1, 3, 2],
       [5, 4, 1, 2, 3],
       [1, 2, 4, 5, 3],
       [5, 2, 1, 3, 4],
       [3, 1, 4, 5, 2],
       [3, 2, 5, 4, 1],
       [2, 5, 1, 3, 4],
       [1, 4, 5, 3, 2],
       [4, 2, 1, 3, 5],
       [5, 2, 4, 1, 3],
       [4, 1, 3, 5, 2],
       [5, 4, 3, 2, 1],
       [5, 4, 1, 2, 3]])

### Check how many students have  the choice that differs from initial one

In [184]:
checkMatch = [False for i in range(n)]
for i in range(n):
    if np.where(res1[i] == 1)[0][0] == np.where(a_ranksTable[i] == 1)[0][0]:
        checkMatch[i] = True

In [185]:
checkMatch

[True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 False]

In [186]:
n - sum(checkMatch)

4

## Second type of constraints

In [187]:
model= Model("Assignment Model")

#Defining the Variable
X = {}
for i in range(n):
    for j in range(m):
        X[i,j] = model.addVar(vtype= GRB.BINARY)
        
d = {}
for i in range(n):
    for j in range(m):
        d[i,j] = model.addVar(vtype= GRB.BINARY)
        
model.setObjective(quicksum(d[i,j] for i in range(n) for j in range(m)), GRB.MINIMIZE)

In [188]:
#Constraint-1
for i in range(n):
     model.addConstr(quicksum(X[i,j] for j in range(m)) <= 1)
#Constraint-2
for j in range(m):
    model.addConstr(quicksum(X[i,j] for i in range(n)) <= upperQ[j])
    
#Constraint-3
for j in range(m):
    model.addConstr(quicksum(X[i,j] for i in range(n)) >= lowerQ[j])        

# for i in range(n):
#     for j in range(m):
#         model.addConstr(quicksum(X[i,k] for k in range(m) if (a_ranks[i][k] <= a_ranks[i][j]))*upperQ[j] \
#                         + quicksum(X[h,j] for h in range(n) if c_score[h][j] > c_score[i][j]) >= upperQ[j])

for i in range(n):
    for j in range(m):
        model.addConstr(quicksum(X[i,k] for k in range(m) if (a_ranks[i][k] <= a_ranks[i][j]))*upperQ[j] \
                        + quicksum(X[h,j] for h in range(n) if c_score[h][j] >= c_score[i][j])\
                        + d[i,j]*upperQ[j] >= upperQ[j])
    
for i in range(n):
    for j in range(m):
        model.addConstr(X[i,j] >= 0)
        
#Constraint-6
for j in range(m):
    for l in range(p):
        model.addConstr(quicksum(X[i,j] for i in range(n) if applicants_types[i] == l+1) <= ub_of_types[j][l])
        
#Constraint-7
for j in range(m):
    for l in range(p):
        model.addConstr(quicksum(X[i,j] for i in range(n) if applicants_types[i] == l+1) >= lb_of_types[j][l])        

In [189]:
model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 315 rows, 250 columns and 2926 nonzeros
Model fingerprint: 0xdd9660be
Variable types: 0 continuous, 250 integer (250 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+00]
Found heuristic solution: objective 64.0000000
Presolve removed 136 rows and 11 columns
Presolve time: 0.01s
Presolved: 179 rows, 239 columns, 2641 nonzeros
Variable types: 0 continuous, 239 integer (239 binary)

Root relaxation: objective 1.000000e+00, 91 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    1.00000    0   18   64.00000    1.00000  98.4%     -    0s
H    0     0                       5.0000000    1.000

In [190]:
res2 = np.array(model.X[:125]).reshape((n,m))
abs(res2)

array([[1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]])

In [191]:
a_ranksTable2 = np.array(a_ranks).reshape((n,m))
a_ranksTable2

array([[2, 5, 1, 4, 3],
       [3, 4, 5, 2, 1],
       [4, 3, 5, 2, 1],
       [1, 3, 2, 5, 4],
       [4, 5, 2, 3, 1],
       [1, 4, 3, 2, 5],
       [2, 5, 4, 1, 3],
       [2, 3, 5, 1, 4],
       [5, 1, 2, 3, 4],
       [3, 5, 1, 4, 2],
       [5, 2, 4, 3, 1],
       [3, 4, 2, 5, 1],
       [4, 5, 1, 3, 2],
       [5, 4, 1, 2, 3],
       [1, 2, 4, 5, 3],
       [5, 2, 1, 3, 4],
       [3, 1, 4, 5, 2],
       [3, 2, 5, 4, 1],
       [2, 5, 1, 3, 4],
       [1, 4, 5, 3, 2],
       [4, 2, 1, 3, 5],
       [5, 2, 4, 1, 3],
       [4, 1, 3, 5, 2],
       [5, 4, 3, 2, 1],
       [5, 4, 1, 2, 3]])

In [192]:
checkMatch2 = [False for i in range(n)]
for i in range(n):
    if np.where(res2[i] == 1)[0][0] == np.where(a_ranksTable2[i] == 1)[0][0]:
        checkMatch2[i] = True

In [193]:
n - sum(checkMatch2)

4

### Define the function

In [147]:
def findAllocation(n, m, p, upperQ, lowerQ, a_ranks, c_score, lb_of_types, ub_of_types, applicants_types, typeConstr=1):
    model= Model("Assignment Model")
    model.Params.LogToConsole = 0
    
    X = {}
    for i in range(n):
        for j in range(m):
            X[i,j] = model.addVar(vtype= GRB.BINARY)
            
    if typeConstr == 1:        

        d = {}
        for i in range(n):
            for j in range(m):
                d[i,j] = model.addVar(lb=0.0, ub=float('inf'), vtype= GRB.CONTINUOUS)

        model.setObjective(quicksum(d[i,j] for i in range(n) for j in range(m)), GRB.MINIMIZE)

        #Constraint-1
        for i in range(n):
            model.addConstr(quicksum(X[i,j] for j in range(m)) <= 1)
        #Constraint-2
        for j in range(m):
            model.addConstr(quicksum(X[i,j] for i in range(n)) <= upperQ[j])

        #Constraint-3
        for j in range(m):
            model.addConstr(quicksum(X[i,j] for i in range(n)) >= lowerQ[j])        

        # for i in range(n):
        #     for j in range(m):
        #         model.addConstr(quicksum(X[i,k] for k in range(m) if (a_ranks[i][k] <= a_ranks[i][j]))*upperQ[j] \
        #                         + quicksum(X[h,j] for h in range(n) if c_score[h][j] > c_score[i][j]) >= upperQ[j])
    
        #Constraint-4
    
        for i in range(n):
            for j in range(m):
                model.addConstr(quicksum(X[i,k] for k in range(m) if (a_ranks[i][k] <= a_ranks[i][j]))*upperQ[j] \
                                + quicksum(X[h,j] for h in range(n) if c_score[h][j] >= c_score[i][j]) + d[i,j] \
                                >= upperQ[j])
                
        #Constraint-5
        for i in range(n):
            for j in range(m):
                model.addConstr(X[i,j] >= 0)
                
        #Constraint-6
        for j in range(m):
            for l in range(p):
                model.addConstr(quicksum(X[i,j] for i in range(n) if applicants_types[i] == l+1) <= ub_of_types[j][l])

        #Constraint-7
        for j in range(m):
            for l in range(p):
                model.addConstr(quicksum(X[i,j] for i in range(n) if applicants_types[i] == l+1) >= lb_of_types[j][l])        
                
                
    else:        
        
        d = {}
        for i in range(n):
            for j in range(m):
                d[i,j] = model.addVar(vtype= GRB.BINARY)

        model.setObjective(quicksum(d[i,j] for i in range(n) for j in range(m)), GRB.MINIMIZE)

        #Constraint-1
        for i in range(n):
            model.addConstr(quicksum(X[i,j] for j in range(m)) <= 1)
        #Constraint-2
        for j in range(m):
            model.addConstr(quicksum(X[i,j] for i in range(n)) <= upperQ[j])

        #Constraint-3
        for j in range(m):
            model.addConstr(quicksum(X[i,j] for i in range(n)) >= lowerQ[j])        

        # for i in range(n):
        #     for j in range(m):
        #         model.addConstr(quicksum(X[i,k] for k in range(m) if (a_ranks[i][k] <= a_ranks[i][j]))*upperQ[j] \
        #                         + quicksum(X[h,j] for h in range(n) if c_score[h][j] > c_score[i][j]) >= upperQ[j])
       
        #Constraint-4
        for i in range(n):
            for j in range(m):
                model.addConstr(quicksum(X[i,k] for k in range(m) if (a_ranks[i][k] <= a_ranks[i][j]))*upperQ[j] \
                                + quicksum(X[h,j] for h in range(n) if c_score[h][j] >= c_score[i][j])\
                                + d[i,j]*upperQ[j] >= upperQ[j])        
        #Constraint-5
        for i in range(n):
            for j in range(m):
                model.addConstr(X[i,j] >= 0)
                
        #Constraint-6
        for j in range(m):
            for l in range(p):
                model.addConstr(quicksum(X[i,j] for i in range(n) if applicants_types[i] == l+1) <= ub_of_types[j][l])

        #Constraint-7
        for j in range(m):
            for l in range(p):
                model.addConstr(quicksum(X[i,j] for i in range(n) if applicants_types[i] == l+1) >= lb_of_types[j][l])        
            
    model.optimize()
    
    res = np.array(model.X[:n*m]).reshape((n,m))
    
    return abs(res)

In [194]:
res3 = findAllocation(n, m, p, upperQ, lowerQ, a_ranks, c_score, lb_of_types, ub_of_types, applicants_types, 2)
res3

array([[1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]])

### Testing

In [196]:
def checkMatch(n, m, res, a_ranks):
    a_ranksTable = np.array(a_ranks).reshape((n,m))
    
    checkMatch = [False for i in range(n)]
    
    for i in range(n):
        if np.where(res[i] == 1)[0][0] == np.where(a_ranksTable[i] == 1)[0][0]:
            checkMatch[i] = True

    return (n - sum(checkMatch))  

In [197]:
checkMatch(n, m, res3, a_ranks)

4

In [222]:
test_size = 50

n_array = np.sort([np.random.randint(25, 100) for i in range(test_size)])
m_array = np.sort([np.random.randint(3, 10) for i in range(test_size)])
p_array = [np.random.randint(1, 4) for i in range(test_size)]

In [223]:
#quotas
upperQ = [list(np.random.randint(int(n_array[i]/m_array[i])+10, \
                                 int(n_array[i]/m_array[i])+50, size=m_array[i])) for i in range(test_size)]

lowerQ = [list(np.random.randint(int(n_array[i]/m_array[i])-10, \
                                 int(n_array[i]/m_array[i]), size=m_array[i])) for i in range(test_size)]

In [224]:
lb_of_types = [[np.random.randint(int(n_array[i]/m_array[i]/p_array[i])-1, \
                                 int(n_array[i]/m_array[i]/p_array[i]), size=p_array[i]) for j in range(m_array[i])] for i in range(test_size)]

ub_of_types = [[np.random.randint(int(n_array[i]/m_array[i])+10, \
                                 int(n_array[i]/m_array[i])+50, size=p_array[i]) for j in range(m_array[i])] for i in range(test_size)]

applicants_types = [np.random.randint(1, p_array[i]+1, size=n_array[i]) for i in range(test_size)]

In [225]:
#Students' ranks of companies
a_ranks = [[random.sample(list(np.arange(m_array[j])+1), m_array[j]) for i in range(n_array[j])] for j in range(test_size)]

#Companies' scores of students
c_score = [np.array([np.random.randint(1, 10, size=n_array[k]) for j in range(m_array[k])]).T for k in range(test_size)]

In [226]:
checkArray1 = []
checkArray2 = []

for i in range(test_size):
#     print(i)
    res1 = findAllocation(n_array[i], m_array[i], p_array[i], upperQ[i], lowerQ[i], a_ranks[i], c_score[i],\
                          lb_of_types[i], ub_of_types[i], applicants_types[i], 1)
    
    checkRes1 = checkMatch(n_array[i], m_array[i], res1, a_ranks[i])
    
    checkArray1.append(checkRes1)
    
    res2 = findAllocation(n_array[i], m_array[i], p_array[i], upperQ[i], lowerQ[i], a_ranks[i], c_score[i], \
                          lb_of_types[i], ub_of_types[i], applicants_types[i], 2)
    
    checkRes2 = checkMatch(n_array[i], m_array[i], res2, a_ranks[i])
    
    checkArray2.append(checkRes2)
    

In [237]:
ind = np.where(np.array(checkArray1) != np.array(checkArray2))[0]
ind

array([20, 30])

In [246]:
checkArray1[20]

8

In [247]:
checkArray2[20]

6

In [248]:
checkArray1[30]

8

In [249]:
checkArray2[30]

7